In [5]:
# import necessary libraries
import pandas as pd
import sys
import os
from sodapy import Socrata  #pull data with Tyler api
import folium

client = Socrata('data.cityofchicago.org',
                 os.environ['CHICAGO_DATA_APP_TOKEN'],  #secrets stay secret, sorry not sorry
                 username="bk2988a@american.edu",
                 password=os.environ['CHICAGO_DATA_PWD'])  #pwds are not for sharing
client.timeout = 50

print('configured')

configured


In [6]:
results = client.get("ijzp-q8t2", limit=10000)  #10,000 takes 7 secs

In [7]:
df = pd.DataFrame.from_records(results)
df['latitude'] = pd.to_numeric(df['latitude'])
df['longitude'] = pd.to_numeric(df['longitude'])

0    41.917874
1    41.837630
2    41.866989
3    41.797415
4    41.896666
Name: latitude, dtype: float64

In [29]:
locations = df[['latitude', 'longitude', 'primary_type']].dropna()
map = folium.Map(location=[locations.latitude.mean(), locations.longitude.mean()], zoom_start=12, control_scale=True)

for index, location in locations.sample(2000).iterrows():
    folium.Marker([location['latitude'], location['longitude']], popup=location['primary_type']).add_to(map)

TypeError: Could not convert 41.79741480841.91787386541.83763030641.86698884141.92253367241.89666601341.78888633941.82691751541.81685550541.75821436741.94575120641.9126909941.9137484941.7190775541.7631082641.92288526941.78694072541.76650821141.88976054841.87384911241.89195134741.85684419141.90159743941.89688858641.79842930341.93317641141.77730986741.90987679741.97722512741.91438362341.88357990641.8041711141.74489949241.76129496741.8853610141.90172919541.75849808841.75462635441.69482962141.91795914241.88542521541.92129109241.74890760941.91176247941.75605583441.77212316641.89070383141.85058391141.88118310441.93349851641.9776564441.87494733841.91863339341.7748844141.77506850741.91809927241.76562119441.79628470641.88186286841.74457097241.69531095641.76185813341.92062586241.76562119441.9975737341.77329076941.70770511841.93975387941.97869097341.91428244741.84931772341.78050335841.83415513141.8308067341.8842819941.79160316641.78021815141.86394152141.85854278941.75447626441.89608313241.7725409